# Using Tweepy to mine data from twitter

guide from https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/

In [1]:
import tweepy
from tweepy import OAuthHandler
import simplejson as json
import config

In [3]:

auth = OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)
 
api = tweepy.API(auth)

In [4]:
def process_or_store(statuses,strname):
    with open(strname,'w') as f:
        for status in statuses:
            print(json.dumps(status._json))
            json.dump(status._json,f)
            f.write('\n')

In [5]:
def tweetfunct(task,amount=10):
    if(task=="htl"):
        statuses = tweepy.Cursor(api.home_timeline).items(amount)
        strname = "home_timeline.json"
    elif(task=="friends"):
        statuses = tweepy.Cursor(api.friends).items()
        strname = "friends.json"
    elif(task=="utl"):
        statuses = tweepy.Cursor(api.user_timeline).items(amount)
        strname = "user_timeline.json"
    else:
        print("no task")
    process_or_store(statuses,strname)

In [6]:
tweetfunct("utl",5)

{"created_at": "Tue Mar 26 03:06:14 +0000 2019", "id": 1110377391052845056, "id_str": "1110377391052845056", "text": "@Fajarbaskoro11 https://t.co/1c9U2IQIkm", "truncated": false, "entities": {"hashtags": [], "symbols": [], "user_mentions": [{"screen_name": "Fajarbaskoro11", "name": "Fajarbaskoro1", "id": 1735836350, "id_str": "1735836350", "indices": [0, 15]}], "urls": [{"url": "https://t.co/1c9U2IQIkm", "expanded_url": "https://d-stv.blogspot.com/2019/03/kak-proposal-dan-project-charter.html", "display_url": "d-stv.blogspot.com/2019/03/kak-pr\u2026", "indices": [16, 39]}]}, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "in_reply_to_status_id": 1107859511967019008, "in_reply_to_status_id_str": "1107859511967019008", "in_reply_to_user_id": 321952904, "in_reply_to_user_id_str": "321952904", "in_reply_to_screen_name": "Steve_Danielz", "user": {"id": 321952904, "id_str": "321952904", "name": "Steve Daniels", "screen_name": "Steve_Danielz", "location":

In [37]:
with open('home_timeline.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4))

{
    "created_at": "Sat Feb 06 06:30:52 +0000 2021",
    "id": 1357939800502792195,
    "id_str": "1357939800502792195",
    "text": "Kebahagiaan tercipta saat penderitaan telah sirna\nPenderitaan muncul ketika kebahagiaan telah berlalu\nBahagia dan k\u2026 https://t.co/0jzG65TJCQ",
    "truncated": true,
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [],
        "urls": [
            {
                "url": "https://t.co/0jzG65TJCQ",
                "expanded_url": "https://twitter.com/i/web/status/1357939800502792195",
                "display_url": "twitter.com/i/web/status/1\u2026",
                "indices": [
                    117,
                    140
                ]
            }
        ]
    },
    "source": "<a href=\"https://mobile.twitter.com\" rel=\"nofollow\">Twitter Web App</a>",
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": nul

In [38]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [39]:
import operator 
from collections import Counter
 
fname = 'home_timeline.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

[('Beasiswa', 5), (',', 5), ('di', 4), ('University', 3), ('of', 3)]


In [43]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('indonesian') + punctuation + ['rt', 'via']

terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]

In [44]:
print(terms_stop)

['https://t.co/ohAR7Q6WYy', 'Beasiswa', 'Musik', 'Royal', 'Academy', 'of', 'Music', 'INGGRIS']


In [45]:
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        # Update the counter
        count_all.update(terms_stop)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

[('Beasiswa', 5), ('University', 3), ('of', 3), ('INGGRIS', 3), ('…', 2)]
